In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
from sklearn.cross_validation import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf
import numpy as np
from sklearn.metrics import roc_curve,auc
from itertools import cycle
from scipy import interp
from sklearn.metrics import confusion_matrix,recall_score,classification_report
import datetime
import time
import warnings
warnings.filterwarnings("ignore")
import itertools
import pickle

/Users/Amelia/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
training_epochs = 200
n_neurons_in_h1 = 340
n_neurons_in_h2 = 128
out_neuron = 10 
learning_rate = 0.01

In [3]:
x = tf.placeholder(tf.float32, name='features')
#y = tf.placeholder(tf.float32, name='labels')

W1 = tf.Variable(tf.truncated_normal([340, 128], stddev=0.1))
b1 = tf.Variable(tf.zeros([128]))
layer1 = tf.nn.relu(tf.matmul(x, W1) + b1)#隐层

W2 = tf.Variable(tf.truncated_normal([128,10], stddev=0.1))
b2 = tf.Variable(tf.zeros([10]))
y_without=tf.matmul(layer1, W2) + b2
y_ = tf.placeholder(tf.float32,shape=[None,10])

weight_=tf.placeholder(tf.float32, name='weight')

In [4]:
y_without.shape

TensorShape([Dimension(None), Dimension(10)])

In [5]:
?tf.nn.weighted_cross_entropy_with_logits

In [154]:
cross_entropy=tf.losses.sigmoid_cross_entropy(multi_class_labels=y_,logits=y_without, weights=weight_)

train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)


In [155]:
def read_my_file_format(filename_queue):

    reader = tf.TextLineReader(skip_header_lines=1)    

    _, value = reader.read(filename_queue)

    record_defaults = list([1.0] for i in range(351))    #这里你有多少列数据就写多少

    row = tf.decode_csv(value,record_defaults=record_defaults)

# col7, col8, col9, col10, col11, co12, col13, col14, col15 ,col16, col17, col18, col19, col20  = tf.decode_csv(value, record_defaults=record_defaults)    

    features = tf.stack(row[11:])
    label = tf.stack(row[0:10])
    weight=tf.stack(row[10:11])

    return features, label, weight

In [156]:
def input_pipeline(batch_size, num_epochs=None):

#     min_after_dequeue = 1000
    capacity = 250

    filename_queue = tf.train.string_input_producer(["train.csv"], num_epochs=num_epochs, shuffle=True)

    example, label, weight = read_my_file_format(filename_queue)     


    example_batch, label_batch, weight_batch = tf.train.shuffle_batch([example, label, weight], 
                                                     batch_size=batch_size, 
                                                     capacity=capacity, 
                                                     min_after_dequeue=100)   
    print(label_batch.shape)
    return example_batch, label_batch, weight_batch

In [157]:
label_batch.shape

(200, 10)

In [158]:
def hit3accu(y_pre,y_test):
    all_pre_list=[]
    for each in y_pre:
        top3_list=[]
        original_prob=each.tolist()
        top3=each[np.argpartition(each,-5)[-5:][::-1]]
        for i in top3:
            top3_list.append(original_prob.index(i))
        all_pre_list.append(top3_list)

    all_test_list=[]
    print(y_test)
    for each in y_test:
        each_list=[]
        freq = each.tolist().count(1)
        if freq==1:
            each_list.append(each.argmax())
        else:
            original_test=each.tolist()
            top2=each[np.argpartition(each,-freq)[-freq:]]
            for i in top2:
                each_list.append(original_test.index(i))
                original_test.remove(i)
        all_test_list.append(each_list)

    calcal=[]
    for each in range(len(all_pre_list)):
        intersection = [v for v in all_pre_list[each] if v in all_test_list[each]]
    #     print(intersection)
        calcal.append(len(intersection)>0)
    return(np.mean(calcal))

In [159]:
test_in_accu=pd.read_csv('test.csv')
X_test_final_accu=test_in_accu.iloc[:,10:]
y_test_accu=test_in_accu.iloc[:,0:10].values
test_in_accu

,d00015,d00049,d00749,d00091,d00088,d00797,d00438,d04283,d00867,d03827,...,tono_2.0,tbtest_0.0,tbtest_2.0,egd_0.0,egd_1.0,egd_2.0,sigcolon_0.0,sigcolon_1.0,sigcolon_2.0,normalAge
0,0,0,1,0,0,0,0,0,0,0,...,1,0,1,0,0,1,1,0,0,1.952991
1,0,1,0,0,1,0,0,0,0,0,...,1,0,1,0,0,1,0,0,1,-0.856800
2,0,0,1,0,0,0,0,0,0,0,...,1,0,1,0,0,1,1,0,0,-0.435331
3,0,0,0,0,1,0,0,0,0,0,...,1,0,1,0,0,1,0,0,1,-0.622650
4,0,1,0,0,0,0,0,0,0,0,...,1,0,1,0,0,1,0,0,1,-0.856800
5,0,1,0,0,1,0,0,0,0,0,...,1,0,1,0,0,1,1,0,0,-0.856800
6,0,1,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,1,0,0,0.969564
7,1,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,1,0,0,-0.856800
8,0,1,1,0,0,0,0,0,0,0,...,1,0,1,0,0,1,1,0,0,-0.716310
9,0,0,0,1,0,0,0,0,0,0,...,1,0,1,0,0,1,1,0,0,0.875905


In [160]:
## 

examples, labels, weights = input_pipeline(200, 10)

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

sess = tf.Session()

# Initialize the variables (like the epoch counter).    
sess.run(init_op)

# merged = tf.summary.merge_all()
# trainwriter =tf.summary.FileWriter("./board/custom", sess.graph)

# Start input enqueue threads.
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)
train_accu=[]
test_accu=[]
train_loss=[]
saver=tf.train.Saver()
now=time.time()
try:
    i = 0;
    while not coord.should_stop():
        i = i + 1
        example_batch, label_batch, weight_batch= sess.run([examples, labels, weights])
#         aa=sess.run(example_batch)
#         print(example_batch)
#         break
        weight_batch_final=weight_batch.flatten()#weight需要展平，https://blog.csdn.net/maoersong/article/details/23823925
        sess.run(train_step, feed_dict={x: example_batch, y_: label_batch, weight_:weight_batch_final})  
        train_loss.append(sess.run(cross_entropy,feed_dict={x:example_batch,y_:label_batch, weight_:weight_batch_final}))
        print("about to enter hit3")
        train_accu.append(hit3accu(sess.run(y,feed_dict={x:example_batch}),label_batch))
        test_accu.append(hit3accu(sess.run(y,feed_dict={x:X_test_final_accu}),y_test_accu))
        if i % 1 == 0:
#             summary = sess.run(merged, feed_dict={x: example_batch, y_: label_batch})
           
            print('iter:%s,train_loss:%s,train_accu:%s,test_accu:%s'%(i,train_loss[-1],train_accu[-1],test_accu[-1]))
#         if i %500==0:
#             model_path = "E:\\pingan\\model\\dataset_filter\\thirty_five\\co-occurence-single\\model{}.ckpt".format(i) 
#             saver.save(sess,model_path)
#             trainwriter.add_summary(summary,i)             
#         print(i)       
#         break
except tf.errors.OutOfRangeError:
    print('Done training -- epoch limit reached')
finally:
    # When done, ask the threads to stop.
    coord.request_stop()

# Wait for threads to finish.
coord.join(threads)
duration=time.time()-now
print(duration)

(200, 10)
about to enter hit3


InvalidArgumentError: You must feed a value for placeholder tensor 'labels_5' with dtype float
	 [[Node: labels_5 = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'labels_5', defined at:
  File "/Users/Amelia/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/Amelia/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/Amelia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/Amelia/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/Amelia/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/Users/Amelia/anaconda3/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 127, in start
    self.asyncio_loo

In [163]:
label_batch.shape

(200, 10)

In [ ]:
test_in=pd.read_csv('test.csv')

In [ ]:
test_in.head()

In [ ]:
y_test=test_in.iloc[:,0:10].values

In [ ]:
X_test_final=test_in.iloc[:,10:]

In [ ]:
y_pre=sess.run(y,feed_dict={x:X_test_final})

In [ ]:
y_test

In [ ]:
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(10):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pre[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_pre.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(10)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(10):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= 10

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

In [ ]:
# Compute macro-average ROC curve and ROC area



# Plot all ROC curves
fig1=plt.figure(figsize=(10,10))
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(10), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
# plt.legend()
plt.legend(loc="lower right")
plt.show()
fig1.savefig('roc.png')

In [ ]:
all_accu=[]
for each in range(len(y_pre)):
    all_accu.append(y_pre[each].argmax()==y_test[each].argmax())
accu_1=(np.mean(all_accu))

In [ ]:
all_pre_list=[]
for each in y_pre:
    top3_list=[]
    original_prob=each.tolist()
    top3=each[np.argpartition(each,-3)[-3:][::-1]]
    for i in top3:
        top3_list.append(original_prob.index(i))
    all_pre_list.append(top3_list)

In [ ]:
all_test_list=[]
for each in y_test:
    each_list=[]
    
    if each.tolist().count(1)==1:
        each_list.append(each.argmax())
    else:
        original_test=each.tolist()
        top2=each[np.argpartition(each,-2)[-2:]]
        for i in top2:
            each_list.append(original_test.index(i))
            original_test.remove(i)
    all_test_list.append(each_list)

In [ ]:
calcal=[]
for each in range(len(all_pre_list)):
    intersection = [v for v in all_pre_list[each] if v in all_test_list[each]]
#     print(intersection)
    calcal.append(len(intersection)>0)
accu_3=(np.mean(calcal))

In [ ]:
all_pre_list_five=[]
for each in y_pre:
    top5_list=[]
    original_prob_five=each.tolist()
    top5=each[np.argpartition(each,-5)[-5:][::-1]]
    for i in top5:
        top5_list.append(original_prob_five.index(i))
    all_pre_list_five.append(top5_list)

In [ ]:
calcal_five=[]
for each in range(len(all_pre_list_five)):
    intersection = [v for v in all_pre_list_five[each] if v in all_test_list[each]]
#     print(intersection)
    calcal_five.append(len(intersection)>0)
accu_5=(np.mean(calcal_five))

In [ ]:
# y_pre=(sess.run(y,feed_dict={x:X_test,y_:y_test}))
# b=(sess.run(y_without,feed_dict={x:X_test,y_:y_test}))

count=[]
for each in y_pre:
    count.append(each.argmax())  #统计每个样本的预测分类
# print(len(count),count.count(10))
# print(sess.run(tf.contrib.metrics.streaming_auc(a,y_test)))
# 2000 2000
test_y=[] #用于计算每类的accuracy
for each in y_test:
    test_y.append(each.argmax())#y_test真实值
#     aa=y_test[each][np.argpartition(y_test[each],-2)[-2:]][::-1]  
#     print(aa)
#     break

In [ ]:
deep_each=[]#存储每类的accuracy
def each_prob_deep(df,n):#该函数用于计算每类的accuracy
    df1=df.loc[df['y_test']==n]
    return np.mean(df1['y_test']==df1['deep_pre'])
df_each=pd.DataFrame({'y_test':test_y,'deep_pre':count})
for each in range(10):
    p=each_prob_deep(df_each,each)
    deep_each.append(p)     #deep预测结果

In [ ]:
f1_score=(classification_report(test_y,count)) #输出f1score
#注意如果是multilabel,test_y应保留multilabel,即每行对应多个1

In [ ]:
print(f1_score)

In [ ]:
#用于存储每类的precision,recall,f1-score,support
class_num=[]
precision=[]
recall=[]
f1_score_=[]
support=[]
lines = f1_score.split('\n')
for each in range(2,35):
    line=lines[each].split()
    class_num.append(line[0])
    precision.append(line[1])
    recall.append(line[2])
    f1_score_.append(line[3])
    support.append(line[4])
#     print(line)

In [ ]:
labels=range(33)##label现在有问题
# value_counts=X_test_final['diag1'].value_counts()
# # print(len(value_counts))
# for each in range(len(value_counts)):
#     labels.append((value_counts.index[each]))#labels的顺序和frequency保持一致

In [ ]:
each_auc=[]#存储每类的auc
for each in labels:
    index=labels.index(each)
    each_auc.append(roc_auc[index])

In [ ]:
#输出并绘制混淆矩阵
def plot_confusion_matrix(y_true,y_predicted,title='Confudion Matrix',cmap=plt.cm.Blues):
    plt.imshow(y_true, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(y_predicted))
    plt.xticks(tick_marks, y_predicted, rotation=0)
    plt.yticks(tick_marks, y_predicted)

    thresh = y_true.max() / 2.
    for i, j in itertools.product(range(y_true.shape[0]), range(y_true.shape[1])):
        plt.text(j, i, y_true[i, j],
                 horizontalalignment="center",
                 color="white" if y_true[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# plt.figure(figsize=10)
#计算混淆矩阵
cnf=confusion_matrix(test_y,count)
np.set_printoptions(precision=2)
# print("Recall metric in the testing dataset: ", cnf[1,1]/(cnf[1,0]+cnf[1,1]))  #计算召回率

# Plot non-normalized confusion matrix
class_names = list(range(33))
fig=plt.figure(figsize=(15,15))
plot_confusion_matrix(cnf,y_predicted=class_names,title='Confusion matrix')
plt.show()
fig.savefig('matrix.png')

In [ ]:
nowTime=datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
df_test=pd.DataFrame()
df_test1=pd.DataFrame()
df_test2=pd.DataFrame()
df_test3=pd.DataFrame()
df_test4=pd.DataFrame()
df_test5=pd.DataFrame()
df_test6=pd.DataFrame()
df_test['Disease']=labels#疾病的名称
df_test2['each_accuracy']=deep_each#每类的accuracy
df_test1['precision']=precision#每类的precision
df_test1['recall']=recall#每类的recall
df_test1['f1_score']=f1_score_#每类的f1-score
df_test1['support']=support#每类的support
df_test['each_auc']=each_auc#每类的auc
df_test3['step_loss']=train_loss#384
df_test3['train_accu']=train_accu#385
df_test4['test_accu']=test_accu#385
df_test5['micro_auc']=[(roc_auc['micro'])]
df_test5['macro_auc']=[(roc_auc['macro'])]
df_test6['precision']=[lines[-2].split()[-4]]
df_test6['recall']=[lines[-2].split()[-3]]
df_test6['f1-score']=[lines[-2].split()[-2]]
df_test6['accu@1']=[accu_1]
df_test6['accu@3']=[accu_3]
df_test6['accu@5']=[accu_5]
writer=pd.ExcelWriter('report_out_'+nowTime+".xlsx",engine='xlsxwriter')
df_out=pd.concat([df_test,df_test1,df_test2,df_test5,df_test6],axis=1)
df_out_all=pd.concat([df_test3,df_test4],axis=1)
# df_out.to_excel('report_out_'+nowTime+".xlsx",index=False,sheet_name='each')
# df_out_all.to_excel('report_out_'+nowTime+'.xlsx',index=False,sheet_name='all')
df_out.to_excel(writer,'each',index=False)
df_out_all.to_excel(writer,'all',index=False)
sheet=writer.book.add_worksheet('matrix')
sheet.insert_image(0,0,'matrix.png')
sheet1=writer.book.add_worksheet('ROC')
sheet1.insert_image(0,0,'roc.png')
writer.save()


In [ ]:
# 保存二进制模型  
output_graph_def = tf.graph_util.convert_variables_to_constants(sess, sess.graph_def, \
                                                                output_node_names=['input','output'])  
with tf.gfile.FastGFile('E:\\pingan\\model\\dataset_filter\\thirty_five\\co-occurence-single\\test.pb', mode='wb') as f:  
    f.write(output_graph_def.SerializeToString())  